In [ ]:

import numpy as np           
# installation de mpl_toolkits pour tracer des frontière (non utilisé pour le moment)
# Si GEOS est installé de manière standard, suivre la procédure suivante sous linux
# Télécharger les sources https://github.com/matplotlib/basemap/archive/refs/tags/v1.3.3.tar.gz
# tar -xvf basemap-1.3.3.tar.gz
# cd basemap-1.3.3/packages/basemap
# python3 -m pip install .
# En cas de problème,  voir https://matplotlib.org/basemap/users/installing.html
# Tout ça est assez lourd
#from mpl_toolkits.basemap import Basemap

import matplotlib.pyplot as plt
from scipy.interpolate import griddata

import plotly.graph_objects as go
import IPython

import gstlearn as gl
from numpy import pi, cos, sin





## Simulation

In [ ]:
gl.variety_define(1,1)

mesh = gl.MeshSpherical()
mesh.reset(None,None,triswitch = "-r5",verbose=False)

X = mesh.getCoordinates(0)
Y = mesh.getCoordinates(1)

db = gl.Db.create()
db["long"]=X
db["long"] = db["long"]-180. #Attention à Mesh barycenter
db["lat"]=Y

db.setLocators(["long","lat"],gl.ELoc.X)

model = gl.Model.createFromParam(gl.ECov.BESSEL_K,range=50,param=1)
S=gl.ShiftOpCs(mesh,model)
whitenoise = gl.ut_vector_simulate_gaussian(mesh.getNApices())
result = gl.VectorDouble(mesh.getNApices())
Q = gl.PrecisionOpCs(S,model.getCova(0),gl.EPowerPT.MINUSHALF)
Q.eval(whitenoise,result)


In [ ]:

def get_lims_colors(surfacecolor):# color limits for a slice
    return np.min(surfacecolor), np.max(surfacecolor)

def colorax(vmin, vmax):
    return dict(cmin=vmin,
                cmax=vmax)


def mapping_map_to_sphere(lon, lat,radius=1.):
    #this function maps the points of coords (lon, lat) to points onto the  sphere of radius radius
    xs=radius*np.cos(lon)*np.cos(lat)
    ys=radius*np.sin(lon)*np.cos(lat)
    zs=radius*np.sin(lat)
    return xs, ys, zs


def d2r(x):
    return x * np.pi / 180.


In [ ]:
nlong = 200
nlat = 100


lon = np.linspace(-np.pi, np.pi, nlong)
lat   = np.linspace(-np.pi/2.,np.pi/2.,  nlat)
lon, lat = np.meshgrid(lon, lat)
lon = lon.T
lat = lat.T
grd = griddata(np.stack([d2r(db["long"]),d2r(db["lat"])])[:,:,0].T,
                np.array(result),
                np.stack([lon,lat]).transpose(1,2,0).reshape(nlong*nlat,2)
              )

xs,ys,zs = mapping_map_to_sphere(lon,lat)
sminz, smaxz = get_lims_colors(grd)

cscale = [[0.0, '#313695'],
 [0.07692307692307693, '#3a67af'],
 [0.15384615384615385, '#5994c5'],
 [0.23076923076923078, '#84bbd8'],
 [0.3076923076923077, '#afdbea'],
 [0.38461538461538464, '#d8eff5'],
 [0.46153846153846156, '#d6ffe1'],
 [0.5384615384615384, '#fef4ac'],
 [0.6153846153846154, '#fed987'],
 [0.6923076923076923, '#fdb264'],
 [0.7692307692307693, '#f78249'],
 [0.8461538461538461, '#e75435'],
 [0.9230769230769231, '#cc2727'],
 [1.0, '#a50026']]

fig = go.Figure(data = [go.Surface(x=xs, y=ys, z=zs, 
                 surfacecolor=grd.reshape(nlong,nlat),colorscale = cscale)]
               )





fig.update_layout(
         title_text='Slices in volumetric data', 
         title_x=0.5,
         width=700,
         height=700,
         coloraxis=dict(
                        colorscale=cscale,
                        colorbar_thickness=25,
                        colorbar_len=0.75,
                        **colorax(sminz, smaxz))
)



In [ ]:
from matplotlib import colors
long = db["long"]
lat = db["lat"]
xi = np.linspace(long.min(), long.max(), 100)
yi = np.linspace(lat.min(), lat.max(), 100)

X,Y = np.meshgrid(xi,yi)


grd = griddata(np.array([db["long"],db["lat"]])[:,::,0].T,
                np.array(result).T,
                np.array([X,Y]).T,
                method='cubic')


In [ ]:
plt.imshow(grd,cmap = "seismic")
plt.colorbar()

In [ ]:
help(colors.ListedColormap)